In this notebook I will try to create a composite map from our canny arrays

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import netCDF4 as nc
import cv2
import datetime
import xarray as xr
import os

%matplotlib inline

#### Importação dos dados de Atlantic-Iberian Biscay Irish- Ocean Physics Reanalysis

Através da command line eu importei uma base de dados relativa a 2019 dos valores de SST de um dado programa do Copernicus. Trata-se de um ficheiro .nc (netCDF)

No entanto no futuro eu vou querer que o meu programa faça download diretamente dos dados num python script

In [3]:
current_path = os.getcwd()
current_path

'C:\\Users\\asus\\GitHub\\JUNO\\notebooks'

In [4]:
from pathlib import Path

data_folder = os.path.join(current_path,"../data")
data_folder

'C:\\Users\\asus\\GitHub\\JUNO\\notebooks\\../data'

In [5]:
nc_path = os.path.join(data_folder, "IBI-2019.nc")
ds = nc.Dataset(nc_path)
data = xr.load_dataset(nc_path)

In [6]:
print(ds)     # informações relativas ao ficheiro netCDF

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF3_CLASSIC data model, file format NETCDF3):
    Conventions: CF-1.0
    source: CMEMS IBI-MFC
    institution: Puertos del Estado (PdE) - Mercator-Ocean (MO)
    references: http://marine.copernicus.eu
    title: CMEMS IBI REANALYSIS: DAILY PHYSICAL PRODUCTS 
    easting: longitude
    northing: latitude
    domain_name: IBI12
    FROM_ORIGINAL_FILE__field_type: mean
    field_date: 20191224
    FROM_ORIGINAL_FILE__longitude_min: -19.f
    FROM_ORIGINAL_FILE__longitude_max: 5.f
    FROM_ORIGINAL_FILE__latitude_min: 26.f
    FROM_ORIGINAL_FILE__latitude_max: 56.f
    z_min: 0.50576f
    z_max: 5698.061f
    contact: mailto: servicedesk.cmems@mercator-ocean.eu
    bulletin_date: 2020-12-01
    bulletin_type: Reanalysis
    _CoordSysBuilder: ucar.nc2.dataset.conv.CF1Convention
    comment: 
    history: Data extracted from dataset http://localhost:8080/thredds/dodsC/cmems_mod_ibi_phy_my_0.083deg-3D_P1D-m
    dimensions(sizes): time(358)

In [7]:
#Converter o ficheiro netCDF para uma dataframe
datadf = data.to_dataframe()
datadf2 = datadf.reset_index()

In [8]:
datadf2 = datadf2.drop(['depth'], axis=1)     #dropar coluna 'depth' (não é necessária pq é sempre igual)
datadf2    #os dados .nc estão agora no formato de uma dataframe

,latitude,time,longitude,thetao
0,35.0,2019-01-01 12:00:00,-19.000000,17.766001
1,35.0,2019-01-01 12:00:00,-18.916666,17.645000
2,35.0,2019-01-01 12:00:00,-18.833334,17.492001
3,35.0,2019-01-01 12:00:00,-18.750000,17.414001
4,35.0,2019-01-01 12:00:00,-18.666666,17.460001
...,...,...,...,...
7320737,45.0,2019-12-24 12:00:00,-5.333334,12.736000
7320738,45.0,2019-12-24 12:00:00,-5.250000,12.770000
7320739,45.0,2019-12-24 12:00:00,-5.166667,13.017000
7320740,45.0,2019-12-24 12:00:00,-5.083334,13.291000


In [9]:
datadf2 = datadf2[['time', 'latitude', 'longitude', 'thetao']]    #reorganizar as colunas da dataframe
datadf2

,time,latitude,longitude,thetao
0,2019-01-01 12:00:00,35.0,-19.000000,17.766001
1,2019-01-01 12:00:00,35.0,-18.916666,17.645000
2,2019-01-01 12:00:00,35.0,-18.833334,17.492001
3,2019-01-01 12:00:00,35.0,-18.750000,17.414001
4,2019-01-01 12:00:00,35.0,-18.666666,17.460001
...,...,...,...,...
7320737,2019-12-24 12:00:00,45.0,-5.333334,12.736000
7320738,2019-12-24 12:00:00,45.0,-5.250000,12.770000
7320739,2019-12-24 12:00:00,45.0,-5.166667,13.017000
7320740,2019-12-24 12:00:00,45.0,-5.083334,13.291000


In [ ]:
#### Começar por tentar fazer um composite map com 2 imagens (canny arrays)

In [10]:
#Df com dados relativos a um dia específico (neste caso 25 Setembro)
dia25 = datadf2[datadf2['time'] == '2019-09-25 12:00:00']
dia25

,time,latitude,longitude,thetao
45123,2019-09-25 12:00:00,35.0,-19.000000,22.527000
45124,2019-09-25 12:00:00,35.0,-18.916666,22.516001
45125,2019-09-25 12:00:00,35.0,-18.833334,22.648001
45126,2019-09-25 12:00:00,35.0,-18.750000,22.809002
45127,2019-09-25 12:00:00,35.0,-18.666666,22.910000
...,...,...,...,...
7305527,2019-09-25 12:00:00,45.0,-5.333334,19.063999
7305528,2019-09-25 12:00:00,45.0,-5.250000,19.021999
7305529,2019-09-25 12:00:00,45.0,-5.166667,18.996000
7305530,2019-09-25 12:00:00,45.0,-5.083334,19.000999


In [11]:
#Df com dados relativos a um dia específico (neste caso 25 Setembro)
dia26 = datadf2[datadf2['time'] == '2019-09-26 12:00:00']
dia26

,time,latitude,longitude,thetao
45292,2019-09-26 12:00:00,35.0,-19.000000,22.477001
45293,2019-09-26 12:00:00,35.0,-18.916666,22.535000
45294,2019-09-26 12:00:00,35.0,-18.833334,22.711000
45295,2019-09-26 12:00:00,35.0,-18.750000,22.859001
45296,2019-09-26 12:00:00,35.0,-18.666666,22.924999
...,...,...,...,...
7305696,2019-09-26 12:00:00,45.0,-5.333334,18.885000
7305697,2019-09-26 12:00:00,45.0,-5.250000,18.834999
7305698,2019-09-26 12:00:00,45.0,-5.166667,18.810001
7305699,2019-09-26 12:00:00,45.0,-5.083334,18.831001


In [12]:
def canny_creation(df, Tmax):
    
    """
    Função que pega numa df com as temperaturas relativas a um certo dia e plota um mapa com as edges definidas
    através do Canny Algorithm
    """
    
    #Pegar na df dum dia e converte-la num np array com os valores de Temperatura para os valores de long e lati
    Temp = df.pivot_table(index='longitude', columns='latitude', values='thetao').T.values
    
    #Converter os valores de temperatura no formato uint8 com valores entre 0-255
    Temp_day = ((Temp - np.nanmin(Temp)) * (1/(np.nanmax(Temp) - np.nanmin(Temp)) * 255)).astype('uint8')

    Temp_day = np.flipud(Temp_day)   #flipud -> Reverse the order of elements along axis 0 (up/down).
    
    #apply the canny algorithm and plot the image with the edges
    canny = cv2.Canny(Temp_day, Tmax/2, Tmax, L2gradient=False, apertureSize=3)
    
    return canny

In [15]:
canny_25 = canny_creation(dia25, 180)
canny_25

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [20]:
canny_26 = canny_creation(dia26, 180)
canny_26.shape

(121, 169)

In [26]:
fp = np.zeros((121,169)).astype('uint8')
fp

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)